# Sharadar Daily Run

# yfinance Daily Run

In [87]:
import requests
import json
import time
import datetime

from pathlib import Path
from dotenv import load_dotenv
import os
import yfinance as yf
import pandas as pd
import requests_cache
import nasdaqdatalink
from requests import Session
from requests_cache import CacheMixin, SQLiteCache
from requests_ratelimiter import LimiterMixin, MemoryQueueBucket
from pyrate_limiter import Duration, RequestRate, Limiter
class CachedLimiterSession(CacheMixin, LimiterMixin, Session):
    pass

session = CachedLimiterSession(
    limiter=Limiter(RequestRate(2, Duration.SECOND*5)),  # max 2 requests per 5 seconds
    bucket_class=MemoryQueueBucket,
    backend=SQLiteCache("yfinance.cache"),
)

load_dotenv()

PROXY_SERVER=os.getenv("PROXY_SERVER")
api_key=os.getenv("NASDAQ_DATA_LINK_API_KEY")
nasdaqdatalink.ApiConfig.api_key = api_key

session = requests_cache.CachedSession('yfinance.cache')
session.headers['User-agent'] = 'moneybot/1.0'


In [88]:
print(api_key)

aTK7iTbaxk3yuo4e-hsu


In [89]:
DATA_STORE = Path('store/assets.h5')

In [91]:
# sp500 = nasdaqdatalink.get_table('SHARADAR/SP500', action='current')
# tickers = sp500['ticker'].tolist()
df = pd.read_csv('STOCK_META_DATA.csv')
df.info()

# Set the ticker column as the index and sort by the index
df.set_index('ticker', inplace=True)
df.sort_index(inplace=True)

tickers = df.index.to_list()
max_length = max(len(s) for s in tickers)
print(max_length)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5494 entries, 0 to 5493
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   ticker     5494 non-null   object 
 1   name       5494 non-null   object 
 2   last_sale  5494 non-null   float64
 3   marketcap  5494 non-null   float64
 4   ipoyear    5494 non-null   float64
 5   sector     5490 non-null   object 
 6   industry   5490 non-null   object 
dtypes: float64(3), object(4)
memory usage: 300.6+ KB
5


In [ ]:
print(tickers)

In [90]:
with pd.HDFStore(DATA_STORE) as store:
    # Remove the specific key if it exists
    if 'yf/minute/us_equity/prices' in store:
        store.remove('yf/minute/us_equity/prices')
        print("Removed existing 'yf/minute/us_equity/prices' from the store")

Removed existing 'yf/minute/us_equity/prices' from the store


In [92]:
with pd.HDFStore(DATA_STORE) as store:
    print(store.info())

<class 'pandas.io.pytables.HDFStore'>
File path: store/assets.h5
/yf/minute/sp500/prices            frame_table  (typ->appendable_multi,nrows->703229,ncols->9,indexers->[index],dc->[datetime,ticker])


In [93]:
def fetch_and_store_history_data():
    sleep_time = 2.5
    batch_start_time = time.time()
    # for t in tickers[n1:n2]:
    with pd.HDFStore(DATA_STORE) as store:
        for t in tickers:
            try:
                # TODO: once up to date, change start to datetime.today - timedelta("1d") (y)
                df = yf.Ticker(t).history(
                   interval="1m",
                   start="2024-05-02",
                   end="2024-05-04",
                   prepost=True,
                   proxy=PROXY_SERVER,
                   keepna=True
                )
                if df.empty:
                    print(f"No data found for {t}")
                    continue
                
                df.index = df.index.tz_localize(None)
                # Add the ticker column
                df['ticker'] = t

                # Set the multi-index
                df.set_index(['ticker', df.index], inplace=True)
                df.index.names = ['ticker', 'datetime']

                # Store the dataframe
                if 'yf/minute/us_equity/prices' not in store:
                    store.put('yf/minute/us_equity/prices', df[:0], format='table', min_itemsize={'ticker': max_length})
                store.append('yf/minute/us_equity/prices', df, format='table', min_itemsize={'ticker': max_length})
                print(f'added {t} to store')
            except Exception as e:
                print(f"Error fetching data for {t}: {str(e)}")
                if "429" in str(e):
                    if sleep_time<3.0:
                        sleep_time += 0.5
                        print("429 error detected, delaying to respect rate limit...")
                    time.sleep(10.0)
            time.sleep(sleep_time)
    batch_end_time = time.time()
    batch_total_time = batch_end_time - batch_start_time
    print(f"Batch ran for {batch_total_time:.2f} seconds")

In [94]:
start_time = time.time()
fetch_and_store_history_data()
end_time = time.time()
total_time = end_time - start_time
print(f"Program ran for {total_time:.2f} seconds")


added A to store
added AA to store
added AACG to store
added AACI to store
added AACT to store
added AADI to store
added AAGR to store
added AAL to store
added AAMC to store
added AAME to store
added AAN to store
added AAOI to store
added AAON to store
added AAP to store
added AAPL to store
added AAT to store
added AB to store
added ABAT to store
added ABBV to store
added ABCB to store
added ABCL to store
added ABEO to store
added ABEV to store
added ABG to store
added ABIO to store
added ABL to store
added ABLV to store
added ABM to store
added ABNB to store
added ABOS to store
added ABR to store
added ABSI to store
added ABT to store
added ABTS to store
added ABUS to store
added ABVC to store
added ABVX to store
added AC to store
added ACA to store


$ACAB: possibly delisted; No price data found  (1m 2024-05-02 -> 2024-05-04)


$ACAB: possibly delisted; No price data found  (1m 2024-05-02 -> 2024-05-04)
No data found for ACAB
added ACAD to store
added ACB to store
added ACCD to store
added ACCO to store
added ACDC to store
added ACEL to store
added ACET to store
added ACGL to store
added ACHC to store
added ACHL to store
added ACHR to store
added ACHV to store
added ACI to store
added ACIC to store
added ACIU to store
added ACIW to store
added ACLS to store
added ACLX to store
added ACM to store
added ACMR to store
added ACN to store
added ACNB to store
added ACNT to store
added ACON to store
added ACR to store
added ACRE to store
added ACRS to store
added ACRV to store
added ACST to store
added ACT to store
added ACTG to store
added ACU to store
added ACVA to store
added ACXP to store
added ADAG to store
added ADAP to store
added ADBE to store
added ADC to store
added ADCT to store
added ADD to store
added ADEA to store
added ADI to store
added ADIL to store
added ADM to store
added ADMA to store
added ADN t

$ADRT: possibly delisted; No price data found  (1m 2024-05-02 -> 2024-05-04)


$ADRT: possibly delisted; No price data found  (1m 2024-05-02 -> 2024-05-04)
No data found for ADRT
added ADSE to store
added ADSK to store
added ADT to store
added ADTH to store
added ADTN to store
added ADTX to store
added ADUS to store
added ADV to store
added ADVM to store
added ADXN to store
added AE to store


...

$AEAE: possibly delisted; No price data found  (1m 2024-05-02 -> 2024-05-04)
No data found for AEAE
added AEE to store
added AEG to store
added AEHL to store
added AEHR to store
added AEI to store
added AEIS to store
added AEM to store
added AEMD to store
added AENT to store
added AEO to store
added AEON to store
added AEP to store
added AER to store
added AERT to store
added AES to store
added AESI to store
added AEVA to store
added AEYE to store
added AEZS to store
added AFAR to store
added AFBI to store
added AFCG to store
added AFG to store


$AFJK: possibly delisted; No price data found  (1m 2024-05-02 -> 2024-05-04)


$AFJK: possibly delisted; No price data found  (1m 2024-05-02 -> 2024-05-04)
No data found for AFJK
added AFL to store
added AFMD to store
added AFRI to store
added AFRM to store
added AFYA to store
added AG to store
added AGAE to store
added AGBA to store
added AGCO to store
added AGEN to store
added AGFY to store
added AGI to store
added AGIO to store
added AGL to store
added AGM to store
added AGMH to store
added AGNC to store
added AGO to store
added AGR to store
added AGRI to store
added AGRO to store
added AGS to store
added AGX to store
added AGYS to store
added AHCO to store
added AHG to store
added AHH to store
added AHR to store
added AHT to store
added AI to store
added AIG to store
added AIHS to store
added AILE to store
added AIM to store
added AIMD to store
added AIN to store
added AINC to store
added AIP to store
added AIR to store
added AIRC to store
added AIRE to store
added AIRG to store
added AIRI to store
added AIRJ to store
added AIRS to store
added AIRT to store
a

$AKO.B: possibly delisted; No timezone found


No data found for AKO.B
added AKR to store
added AKRO to store
added AKTS to store
added AKTX to store
added AKYA to store
added AL to store
added ALAB to store
added ALAR to store
added ALB to store
added ALBT to store
added ALC to store
added ALCE to store
added ALCO to store
added ALCY to store
added ALDX to store
added ALE to store
added ALEC to store
added ALEX to store
added ALG to store
added ALGM to store
added ALGN to store
added ALGS to store
added ALGT to store
added ALHC to store
added ALIM to store
added ALIT to store
added ALK to store
added ALKS to store
added ALKT to store
added ALL to store
added ALLE to store
added ALLG to store
added ALLK to store
added ALLO to store
added ALLR to store
added ALLT to store
added ALLY to store
added ALNT to store
added ALNY to store
added ALOT to store
added ALPP to store
added ALRM to store
added ALRN to store
added ALRS to store
added ALSA to store
added ALSN to store
added ALT to store
added ALTG to store
added ALTI to store
added 

# Alpaca Daily Run